In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df_news = pd.read_csv('dataset/news/AAPL_main.csv')
df_rela = pd.read_csv('dataset/news/AAPL_relation.csv')

In [3]:
df_aapl = pd.read_csv('dataset/stocks/AAPL.csv')

In [4]:
df_news_t = df_rela[df_rela['ticker'] == df_rela['source_ticker']]
df_news_t['time'] = pd.to_datetime(df_news_t['time']).dt.tz_localize("UTC") \
                    .dt.tz_convert("America/New_York")
df_news_t['sentiment'] = df_news_t['sentiment'].map(
    {'neutral':0, 'positive':1, 'negative':-4.22} # maybe
)
df_news_t['date'] = df_news_t['time'].dt.date

/var/folders/pj/znl5r0ln0hv_g1yjdthtr3f40000gn/T/ipykernel_3662/3384407807.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news_t['time'] = pd.to_datetime(df_news_t['time']).dt.tz_localize("UTC") \
/var/folders/pj/znl5r0ln0hv_g1yjdthtr3f40000gn/T/ipykernel_3662/3384407807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_news_t['sentiment'] = df_news_t['sentiment'].map(
/var/folders/pj/znl5r0ln0hv_g1yjdthtr3f40000gn/T/ipykernel_3662/3384407807.py:7: SettingWithCopyWarning: 
A value is trying to 

In [5]:
s_senti = df_news_t.groupby('date')['sentiment'].sum().reset_index()

In [6]:
df_aapl_t = df_aapl[df_aapl['time'] >= "2024-07-02"]
df_aapl_t['time'] = pd.to_datetime(df_aapl_t['time']).dt.tz_localize("UTC") \
                    .dt.tz_convert("America/New_York")
df_aapl_t['change'] = df_aapl['close'].diff()
df_aapl_t['date'] = df_aapl_t['time'].dt.date

/var/folders/pj/znl5r0ln0hv_g1yjdthtr3f40000gn/T/ipykernel_3662/3429688966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl_t['time'] = pd.to_datetime(df_aapl_t['time']).dt.tz_localize("UTC") \
/var/folders/pj/znl5r0ln0hv_g1yjdthtr3f40000gn/T/ipykernel_3662/3429688966.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aapl_t['change'] = df_aapl['close'].diff()
/var/folders/pj/znl5r0ln0hv_g1yjdthtr3f40000gn/T/ipykernel_3662/3429688966.py:5: SettingWithCopyWarning: 
A value is trying to be set 

In [7]:
s_stock = df_aapl_t[['date', 'change']].reset_index(drop=True)

In [8]:
s_corr = pd.merge(s_stock, s_senti, on="date", how="left")
s_corr['sentiment'] = s_corr['sentiment'].fillna(0)

In [9]:
stats.pearsonr(s_corr['change'], s_corr['sentiment'])

PearsonRResult(statistic=0.38455360070978467, pvalue=0.00030430175142348305)

In [10]:
s_corr['change_c'] = (s_corr['change'] > 0).astype(int)
s_corr['sentiment_c'] = (s_corr['sentiment'] > 0).astype(int)

In [11]:
stats.pearsonr(s_corr['change_c'], s_corr['sentiment_c'])

PearsonRResult(statistic=0.16581183399153315, pvalue=0.13171452155004842)

In [12]:
split_index = int(len(s_corr) * 0.8)
s_corr_train = s_corr.iloc[:split_index] 
s_corr_test = s_corr.iloc[split_index:]

In [13]:
model = LinearRegression()
model.fit(s_corr_train[['sentiment']], s_corr_train['change'])
model.coef_[0], model.intercept_

(0.24046318367420955, 0.1894072501006392)

In [14]:
pred_train = model.predict(s_corr_train[['sentiment']])
mse_train = mean_squared_error(pred_train, s_corr_train['change'])
pred_test = model.predict(s_corr_test[['sentiment']])
mse_test = mean_squared_error(pred_test, s_corr_test['change'])
np.sqrt(mse_train), np.sqrt(mse_test)

(2.9979050630998, 2.3456261366752464)

In [15]:
np.std(s_corr_train['change']), np.std(s_corr_test['change'])

(3.2120300922365024, 2.691315136253978)